In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
import seaborn as sns

In [69]:
test = pd.read_csv('/Users/Masa/anaconda/titanic_test.csv')
train = pd.read_csv('/Users/Masa/anaconda/titanic_train.csv')

In [70]:
# complement Age of TRAIN data
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
indexer = train['Age'].isnull()
lr.fit(train.loc[~indexer, ['Pclass', 'SibSp', 'Parch', 'Fare']], train.loc[~indexer, 'Age'])
predicted = lr.predict(train.loc[indexer, ['Pclass', 'SibSp', 'Parch', 'Fare']])
train.loc[indexer, 'Age'] = predicted
# complement Fare of TEST data 
indexer_fare = test['Fare'].isnull()
test.loc[indexer_fare, 'Fare'] = [0]

In [71]:
# complement Age of TEST data
indexer_ = test['Age'].isnull()
lr.fit(test.loc[~indexer_, ['Pclass', 'SibSp', 'Parch', 'Fare']], test.loc[~indexer_, 'Age'])
predicted_ = lr.predict(test.loc[indexer_, ['Pclass', 'SibSp', 'Parch', 'Fare']])
test.loc[indexer_, 'Age'] = predicted_

In [72]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64

In [73]:
train.Sex = train.Sex.replace(['male','female'],[0,1])
train.Embarked = train.Embarked.replace(['C','S','Q'],[0,1,2])
train = train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis = 1)

In [74]:
# complement Embarked of TRAIN data
indexer2 = train['Embarked'].isnull()
train.loc[indexer2, 'Embarked'] = [0,0]
# complement

In [75]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64

In [76]:
# confirm that there are no NaNs
train.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [77]:
# hold-out
from sklearn.model_selection import train_test_split

train_survived = train['Survived']
train_data = train.drop(['Survived'], axis= 1)
train_train_data, test_train_data, train_train_survived, test_train_survived = train_test_split(train_data, train_survived, test_size = 0.3, random_state=0)

In [78]:
from sklearn.tree import DecisionTreeClassifier
tree_1 = DecisionTreeClassifier(random_state=0)
tree_1.fit(train_train_data, train_train_survived)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=0, splitter='best')

In [79]:
from sklearn.tree import export_graphviz
export_graphviz(tree_1, out_file='tree_1.dot', feature_names=['Pclass', 'Sex', 'Age', 'SibSp', 'ParCh','Fare','Embarked'], class_names=['Survived', 'Not survived'], impurity = False, filled =True)

In [80]:
from sklearn.ensemble import RandomForestClassifier
rfc_1 = RandomForestClassifier(random_state = 0, n_estimators=10)
rfc_1.fit(train_train_data, train_train_survived)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [81]:
print('dtc train: {:.3f}'.format(tree_1.score(train_train_data, train_train_survived)))
print('dtc test: {:.3f}'.format(tree_1.score(test_train_data, test_train_survived)))
print('rfc train: {:.3f}'.format(rfc_1.score(train_train_data, train_train_survived)))
print('rfc test: {:.3f}'.format(rfc_1.score(test_train_data, test_train_survived)))

dtc train: 0.979
dtc test: 0.780
rfc train: 0.963
rfc test: 0.821


In [82]:
test.Sex = test.Sex.replace(['male','female'],[0,1])
test.Embarked = test.Embarked.replace(['C','S','Q'],[0,1,2])
test = test.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis = 1)

In [83]:
test.isnull().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [84]:
test_predict = rfc_1.predict(test)

In [85]:
df_out = pd.read_csv('/Users/Masa/anaconda/titanic_test.csv')
df_out['Survived']= test_predict

In [86]:
df_out[['PassengerId', 'Survived']].to_csv('/Users/Masa/anaconda/titanic_submit.csv', index = False)